In [1]:
# -- coding: utf-8 --
import numpy as np
import pandas as pd
import datetime
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import  Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import RidgeClassifierCV,LogisticRegression
from sklearn.model_selection import KFold
import lightgbm as lgb
from sklearn.preprocessing import MaxAbsScaler,StandardScaler
from sklearn.svm import LinearSVR
from tpot.builtins import StackingEstimator
from sklearn.ensemble import BaggingClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor,XGBClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
#自定义分类器
u'''
融合lgb和LR
利用lgb生成新特征，再用新特征one-hot编码，最后利用LR左分类预测。
'''
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import xgboost as xgb
color = sns.color_palette()
import lightgbm as lgb

%matplotlib inline

import time
import datetime
import numpy as np
import pandas as pd
import lightgbm as lgb
from dateutil.parser import parse
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train=pd.read_csv('./train.csv')
test=pd.read_csv('./test.csv')

together=pd.concat([train,test],axis=0)
together['tijian_date']=pd.to_datetime(together['tijian_date'])


together['month'] = together['tijian_date'].apply(lambda x: x.month)
together['day'] = together['tijian_date'].apply(lambda x: x.day)

def is_weekday(x):
    wd=x.weekday()
    return wd
together['weekday'] = together['tijian_date'].apply(is_weekday)
together.drop(['tijian_date'],inplace=True,axis=1)
together.drop(['id'],inplace=True,axis=1)
labels=together.tang[:5642]
together.drop(['tang'],inplace=True,axis=1)
#together['red_two']=together['redcell_count']*together['redcell_count']
#together['gysz_two']=together['gysz']*together['gysz']
#together['jxls_m_two']=together['jxls_m']*together['jxls_m']
#together['ns_two']=together['ns']*together['ns']
#together['hxb_avgxhdbnd_two']=together['hxb_avgxhdbnd']*together['hxb_avgxhdbnd']
def chang_sex(x):
    if str(x)=='1':
        return int(1)
    else:
        return int(0)
together['sex']=together.sex.apply(chang_sex)
together_fill=together.fillna(together.median(axis=0))
#together_fill['res_plus']=together_fill['age']*together_fill['redcell_count']
#together_fill['mid_cell_div']=together_fill['zxl_cell']*1.0/together_fill['linba_cell']
#train_feat=together_fill[:5642]
#test_feat=together_fill[5642:]
#train_feat.drop('id',inplace=True,axis=1)
#test_id=test_feat['id']
#test_feat.drop('id',inplace=True,axis=1)

In [3]:
together_copy=together_fill.copy()

In [4]:
def reverse_one(x):
    if x==0:
        return 1e8
    else:
        return 1.0/x

In [5]:
for i in together_copy.columns.values:
    together_copy[i+'_onedivide']=together_copy[i].apply(reverse_one)

In [6]:
poly = preprocessing.PolynomialFeatures(degree=2, interaction_only=True)
together_res=poly.fit_transform(together_copy)

In [7]:
len(together_copy.columns)

84

In [12]:
coll=zip(poly.get_feature_names()[1:85],together_copy.columns)

In [13]:
together_res

array([[  1.00000000e+00,   4.10000000e+01,   2.31000000e+01, ...,
          8.33333333e-03,   1.38888889e-02,   1.66666667e-02],
       [  1.00000000e+00,   4.10000000e+01,   3.62500000e+01, ...,
          5.26315789e-03,   3.33333333e-02,   1.75438596e-02],
       [  1.00000000e+00,   4.60000000e+01,   1.52300000e+01, ...,
          3.84615385e-03,   3.33333333e-02,   1.28205128e-02],
       ..., 
       [  1.00000000e+00,   7.90000000e+01,   1.32400000e+01, ...,
          5.00000000e-03,   2.50000000e-02,   1.25000000e-02],
       [  1.00000000e+00,   3.90000000e+01,   7.08000000e+00, ...,
          5.00000000e-03,   2.50000000e-02,   1.25000000e-02],
       [  1.00000000e+00,   5.70000000e+01,   2.14650000e+01, ...,
          5.00000000e-03,   2.50000000e-02,   1.25000000e-02]])

In [14]:
new_poly=pd.DataFrame()
for ix,i in enumerate(poly.get_feature_names()):
    new_poly[i]=together_res[:,ix]

In [29]:
train_feat=new_poly[:5642]
test_feat=new_poly[5642:]

In [34]:
param={
   'colsample_bytree':0.8,
    'subsample':0.8,
    'objective':'reg:tweedie',
    # 'eval_metric':'rmse',
    'tweedie_variance_power':1.4,
    'eta':0.008,
    'max_depth':5,
    'booster':'gbtree',
    'gamma': 1,
    'silent': 1,
    'min child weight':5,
    'eval_metric':'rmse',
    'tree_method':'gpu_hist'
}

In [35]:
dtrain=xgb.DMatrix(train_feat,label=labels)

In [37]:
gpu_res={}
model=xgb.train(param,dtrain,num_boost_round=4000,evals=[(dtrain,'train')],evals_result=gpu_res,verbose_eval=1000,)

[0]	train-rmse:4.25701
[1000]	train-rmse:0.651414
[2000]	train-rmse:0.43427
[3000]	train-rmse:0.293559
[3999]	train-rmse:0.202543


In [39]:
score=model.get_fscore()

In [41]:
ss=[(k,v) for k,v in score.iteritems()]

In [45]:
ss.sort(key=lambda x:x[1],reverse=True)

In [46]:
ss

[('x0', 2700),
 ('x1', 1578),
 ('x2', 1019),
 ('x4', 663),
 ('x3', 613),
 ('x9', 432),
 ('x5', 424),
 ('x6', 359),
 ('x7', 336),
 ('x9 x12', 325),
 ('x11', 265),
 ('x12', 265),
 ('x0 x25', 256),
 ('x10', 252),
 ('x13', 248),
 ('x8', 244),
 ('x10 x12', 240),
 ('x16', 225),
 ('x9 x53', 225),
 ('x14 x63', 214),
 ('x18 x63', 208),
 ('x14', 196),
 ('x0 x40', 173),
 ('x0 x59', 172),
 ('x17', 170),
 ('x13 x63', 166),
 ('x0 x49', 165),
 ('x1 x48', 164),
 ('x15', 162),
 ('x2 x12', 161),
 ('x0 x41', 155),
 ('x12 x63', 154),
 ('x11 x54', 152),
 ('x5 x79', 151),
 ('x0 x6', 151),
 ('x25', 150),
 ('x16 x25', 149),
 ('x23', 148),
 ('x0 x83', 147),
 ('x0 x67', 147),
 ('x21 x60', 144),
 ('x0 x16', 142),
 ('x0 x15', 142),
 ('x22 x65', 142),
 ('x0 x46', 140),
 ('x0 x38', 139),
 ('x0 x8', 138),
 ('x20 x63', 136),
 ('x4 x25', 136),
 ('x1 x66', 135),
 ('x11 x39', 134),
 ('x0 x28', 134),
 ('x28 x30', 133),
 ('x51 x54', 131),
 ('x2 x53', 131),
 ('x17 x60', 130),
 ('x11 x12', 129),
 ('x2 x9', 128),
 ('x36 x39'

In [47]:
coll   ('x9 x12', 325     ('x0 x25', 256),    ('x10 x12', 240),   ('x9 x53', 225),     ('x14 x63', 214),    ('x18 x63', 208),

[('x0', 'age'),
 ('x1', 'basajzh_m'),
 ('x2', 'bdb'),
 ('x3', 'bq_rate'),
 ('x4', 'dh_cell'),
 ('x5', 'dmdzdbdgc'),
 ('x6', 'gaxjzh_m'),
 ('x7', 'gmdzdbdgc'),
 ('x8', 'gysz'),
 ('x9', 'hxb_avgtj'),
 ('x10', 'hxb_avgxhdbl'),
 ('x11', 'hxb_avgxhdbnd'),
 ('x12', 'hxb_tjfbkd'),
 ('x13', 'hxb_yj'),
 ('x14', 'jg'),
 ('x15', 'jxls_m'),
 ('x16', 'linba_cell'),
 ('x17', 'ns'),
 ('x18', 'ns.1'),
 ('x19', 'qdb'),
 ('x20', 'redcell_count'),
 ('x21', 'sex'),
 ('x22', 'shijian_cell'),
 ('x23', 'shisuan_cell'),
 ('x24', 'tmdasajzh_m'),
 ('x25', 'whitecell_count'),
 ('x26', 'xhdb'),
 ('x27', 'xxb_avgtj'),
 ('x28', 'xxb_biji'),
 ('x29', 'xxb_count'),
 ('x30', 'xxb_tjfbkd'),
 ('x31', 'yg_bmkt'),
 ('x32', 'yg_bmky'),
 ('x33', 'yg_ekt'),
 ('x34', 'yg_eky'),
 ('x35', 'yg_hxkt'),
 ('x36', 'zdb'),
 ('x37', 'zdgc'),
 ('x38', 'zxl_cell'),
 ('x39', 'month'),
 ('x40', 'day'),
 ('x41', 'weekday'),
 ('x42', 'age_onedivide'),
 ('x43', 'basajzh_m_onedivide'),
 ('x44', 'bdb_onedivide'),
 ('x45', 'bq_rate_onedivide'),